<h3>라이브러리</h3>

In [1]:
import pandas as pd
import re
import torch
#쿠레 설치 및 import
#! pip install -U sentence-transformers faiss-cpu
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

c:\Users\mj120\OneDrive\Desktop\3학년 2학기\SW캡스톤디자인\데이터\paneldata(피앰아이)\Quickpoll\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3>함수</h3>

In [2]:
#2차원에서 비교
def embeding(emb_list,user_emb):
    #문항 벡터 평균
    #(N,D)  /dim=0(평균 축) << 임베딩의 동일 차원(feature) ,=1 <<정보손실
    rep_emb = torch.stack(emb_list, dim=0).mean(dim=0)
    #만약을 위하여
    if rep_emb.dim() == 1:
        rep_emb = rep_emb.unsqueeze(0)

    # 평균 후 정규화 / 코사인 값 -1~1 보장 /
    # dim=1 각 행마다 정규화(파이토치) 유클리드거리 p=2, dim=1 =벡터의 길이를 1로 변환
    rep_emb = F.normalize(rep_emb, p=2, dim=1, eps=1e-12)

    # 사용자 질의 차원 정리 (D,) → (1, D)
    if user_emb.dim() == 1:  
        user_emb = user_emb.unsqueeze(0)

    # dim=1 각 행마다 정규화(파이토치) 유클리드거리 p=2, dim=1 =벡터의 길이를 1로 변환
    user_emb = F.normalize(user_emb, p=2, dim=1, eps=1e-12)

    # dim=1 행
    scores = F.cosine_similarity(rep_emb, user_emb, dim=1)
    scores = scores.clamp(-1, 1)  # 안정성 확보

    k = min(5, scores.numel())
    topk = torch.topk(scores, k=k, largest=True)

    return topk

<h3>데이터 로드</h3>

In [3]:
person_outer=pd.read_csv("full_outer_loyalty.csv")

C:\Users\mj120\AppData\Local\Temp\ipykernel_21472\1533138882.py:1: DtypeWarning: Columns (3,10,11,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  person_outer=pd.read_csv("full_outer_loyalty.csv")


<h3>데이터 업데이트</h3>

In [ ]:
# #title1
# title={
#     문장의 문맥을 살리는 결합 방식
#     모든 불용어가 제거된 질문 문항 + 선지 
# }

In [ ]:
# #title2
# title={
#     문항과 선지의 관계를 살리는 결합 방식
#     모든 불용어가 제거된 질문 문항 + [SEP] + 선지
# }

In [ ]:
# # #title3
# title={
#     문항과 선지의 관계를 살리는 결합 방식
#     최소한의 불용어가 제거된 질문 문항 + [SEP] + 선지
# }

In [ ]:
# #title4
# title={
#     문항과 선지의 관계를 살리는 결합 방식
#     온전한 질문 문항 + [SEP] + 선지
# }

In [8]:
for qnum, qtext in title.items():
    if qnum in person_outer.columns:
        person_outer[qnum] = person_outer[qnum].apply(
            lambda v: f"{qtext}{v}" if pd.notna(v) else v
        )

In [9]:
# for qnum, qtext in title.items():
#     if qnum in person_outer.columns:
#         person_outer[qnum] = person_outer[qnum].apply(
#             lambda v: f"{qtext}[SEP]{v}" if pd.notna(v) else v
#         )

<h3>모델 호출</h3>

In [ ]:
model = SentenceTransformer("nlpai-lab/KURE-v1")

<h3>샘플 추출</h3>

In [11]:
sample=person_outer.head(100)

In [12]:
sample1 = sample[['문항21', '문항26_1', '문항26_2', '문항29']]

<h3>#mental health</h3>

In [ ]:
qp21_emb = model.encode(
    sample1['문항21'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
qp26_1_emb = model.encode(
    sample1['문항26_1'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
qp26_2_emb = model.encode(
    sample1['문항26_2'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
qp29_emb = model.encode(
    sample1['문항29'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
user_emb = model.encode(
    "돈으로 스트레스를 많이 느끼는 사람",
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)

In [ ]:
emb_list = [qp21_emb,qp26_1_emb,qp26_2_emb,qp29_emb]

topk = embeding(emb_list,user_emb)

top_idx = topk.indices.detach().cpu().numpy()
top_scores = topk.values.detach().cpu().numpy()

result = sample1.iloc[top_idx].copy()
result["similarity"] = top_scores

print(result)

<h3>#Offline & Travel</h3>

In [16]:
sample2 = sample[["문항1","문항9","문항23"]]

In [17]:
qp1_emb = model.encode(
    sample2['문항1'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
qp9_emb = model.encode(
    sample2['문항9'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
qp23_emb = model.encode(
    sample2['문항23'],
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)
user_emb = model.encode(
    "여행을 좋아하는 사람",
    batch_size=64,
    convert_to_tensor=True,     
    normalize_embeddings=True 
)

In [ ]:
emb_list = [qp1_emb,qp9_emb,qp23_emb]

topk = embeding(emb_list,user_emb)

top_idx = topk.indices.detach().cpu().numpy()
top_scores = topk.values.detach().cpu().numpy()

result = sample2.iloc[top_idx].copy()
result["similarity"] = top_scores

print(result)

<h3>비유사 카테고리 결과</h3>

In [ ]:
emb_list = [qp21_emb,qp26_1_emb,qp26_2_emb,qp29_emb]

topk = embeding(emb_list,user_emb)

top_idx = topk.indices.detach().cpu().numpy()
top_scores = topk.values.detach().cpu().numpy()

result = sample1.iloc[top_idx].copy()
result["similarity"] = top_scores

print(result)